In [367]:
import numpy as np              
import matplotlib.pyplot as plt 
%matplotlib inline

import pandas as pd             
import seaborn as sns          
import sklearn

# 0. Классификация грибов
Допустим вы хотите узнать ядовит ли найденный вами гриб, но вы плохо разбираетесь в этом. С помощью описанния гриба будем определить является ли он съедобным.
Воспользуемся для этого слудующим датасетом:
# 1. Чтение данных

In [368]:
url = "https://raw.githubusercontent.com/Alex-Sergeev/Machine-Learning-2020/main/mushrooms.csv"
dataset = pd.read_csv(url)

In [369]:
dataset.shape

(8124, 23)

# 2. Визуализация данных и т.д.
Проверим правильность загрузки данных

In [370]:
dataset.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [371]:
dataset.tail()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
8119,e,k,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,b,c,l
8120,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,n,o,p,b,v,l
8121,e,f,s,n,f,n,a,c,b,n,...,s,o,o,p,o,o,p,b,c,l
8122,p,k,y,n,f,y,f,c,n,b,...,k,w,w,p,w,o,e,w,v,l
8123,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,o,c,l


Для сокращения в изначальном датасете все признаки иммеют сокращения и имеют следующие значения:

* Главный атрибут class: съедобен = e, ядовит = p
* cap-shape: bell=b, conical=c, convex=x, flat=f, knobbed=k, sunken=s
* cap-surface: fibrous=f, grooves=g, scaly=y, smooth=s
* cap-color: brown=n, buff=b, cinnamon=c, gray=g, green=r, pink=p, purple=u, red=e, white=w, yellow=y
* bruises: bruises=t, no=f
* odor: almond=a, anise=l, creosote=c, fishy=y, foul=f, musty=m, none=n, pungent=p, spicy=s
* gill-attachment: attached=a, descending=d, free=f, notched=n
* gill-spacing: close=c, crowded=w, distant=d
* gill-size: broad=b, narrow=n
* gill-color: black=k, brown=n, buff=b, chocolate=h, gray=g, green=r, orange=o, pink=p, purple=u, red=e, white=w, yellow=y
* stalk-shape: enlarging=e, tapering=t
* stalk-root: bulbous=b, club=c, cup=u, equal=e, rhizomorphs=z, rooted=r, missing=?
* stalk-surface-above-ring: fibrous=f, scaly=y, silky=k, smooth=s
* stalk-surface-below-ring: fibrous=f, scaly=y, silky=k, smooth=s
* stalk-color-above-ring: brown=n, buff=b, cinnamon=c, gray=g, orange=o, pink=p, red=e, white=w, yellow=y
* stalk-color-below-ring: brown=n, buff=b, cinnamon=c, gray=g, orange=o, pink=p, red=e, white=w, yellow=y
* veil-type: partial=p, universal=u
* veil-color: brown=n, orange=o, white=w, yellow=y
* ring-number: none=n, one=o, two=t
* ring-type: cobwebby=c, evanescent=e, flaring=f, large=l, none=n, pendant=p, sheathing=s, zone=z
* spore-print-color: black=k, brown=n, buff=b, chocolate=h, green=r, orange=o, purple=u, white=w, yellow=y
* population: abundant=a, clustered=c, numerous=n, scattered=s, several=v, solitary=y
* habitat: grasses=g, leaves=l, meadows=m, paths=p, urban=u, waste=w, woods=d

Задача предсказать съедобен гриб или нет.

Так как признак ring-number по сути является количественным сразу же изменим его, а так же посчитаем основные характеристики(какие возможны):

In [429]:
dataset['ring-number'] = pd.factorize(dataset['ring-number'])[0]
dataset['ring-number'].head()

0    0
1    0
2    0
3    0
4    0
Name: ring-number, dtype: int64

In [373]:
dataset.describe()

,ring-number
count,8124.000000
mean,0.082718
std,0.291116
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,2.000000


In [374]:
dataset.describe(include = ['object'])

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-type,spore-print-color,population,habitat
count,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,...,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124
unique,2,6,4,10,2,9,2,2,2,12,...,4,4,9,9,1,4,5,9,6,7
top,e,x,y,n,f,n,f,c,b,b,...,s,s,w,w,p,w,p,w,v,d
freq,4208,3656,3244,2284,4748,3528,7914,6812,5612,1728,...,5176,4936,4464,4384,8124,7924,3968,2388,4040,3148


Так как номинальный признак один построить матрицу корреляций не является возможным.

Проверим какие значения принимают категориальные признаки и есть ли пропущенные значения:

In [375]:
dataset['cap-shape'].value_counts()

x    3656
f    3152
k     828
b     452
s      32
c       4
Name: cap-shape, dtype: int64

In [376]:
dataset['cap-surface'].value_counts()

y    3244
s    2556
f    2320
g       4
Name: cap-surface, dtype: int64

In [377]:
dataset['cap-color'].value_counts()

n    2284
g    1840
e    1500
y    1072
w    1040
b     168
p     144
c      44
r      16
u      16
Name: cap-color, dtype: int64

In [378]:
dataset['bruises'].value_counts()

f    4748
t    3376
Name: bruises, dtype: int64

In [379]:
dataset['odor'].value_counts()

n    3528
f    2160
y     576
s     576
l     400
a     400
p     256
c     192
m      36
Name: odor, dtype: int64

In [380]:
dataset['gill-attachment'].value_counts()

f    7914
a     210
Name: gill-attachment, dtype: int64

In [381]:
dataset['gill-spacing'].value_counts()

c    6812
w    1312
Name: gill-spacing, dtype: int64

In [382]:
dataset['gill-size'].value_counts()

b    5612
n    2512
Name: gill-size, dtype: int64

In [383]:
dataset['gill-color'].value_counts()

b    1728
p    1492
w    1202
n    1048
g     752
h     732
u     492
k     408
e      96
y      86
o      64
r      24
Name: gill-color, dtype: int64

In [384]:
dataset['stalk-shape'].value_counts()

t    4608
e    3516
Name: stalk-shape, dtype: int64

In [385]:
dataset['stalk-root'].value_counts()

b    3776
?    2480
e    1120
c     556
r     192
Name: stalk-root, dtype: int64

In [386]:
dataset['stalk-surface-above-ring'].value_counts()

s    5176
k    2372
f     552
y      24
Name: stalk-surface-above-ring, dtype: int64

In [387]:
dataset['stalk-surface-below-ring'].value_counts()

s    4936
k    2304
f     600
y     284
Name: stalk-surface-below-ring, dtype: int64

In [388]:
dataset['stalk-color-above-ring'].value_counts()

w    4464
p    1872
g     576
n     448
b     432
o     192
e      96
c      36
y       8
Name: stalk-color-above-ring, dtype: int64

In [389]:
dataset['stalk-color-below-ring'].value_counts()

w    4384
p    1872
g     576
n     512
b     432
o     192
e      96
c      36
y      24
Name: stalk-color-below-ring, dtype: int64

In [390]:
dataset['veil-type'].value_counts()

p    8124
Name: veil-type, dtype: int64

Неожиданно в данном датасете остутсвует различие в данном признаке несмотря на то что он был заявлен.

Удалим данным признак из таблицы ввиду его бессмысленности.

In [391]:
dataset = dataset.drop(['veil-type'], axis = 1)

In [392]:
dataset['veil-color'].value_counts()

w    7924
n      96
o      96
y       8
Name: veil-color, dtype: int64

In [393]:
dataset['ring-type'].value_counts()

p    3968
e    2776
l    1296
f      48
n      36
Name: ring-type, dtype: int64

In [394]:
dataset['spore-print-color'].value_counts()

w    2388
n    1968
k    1872
h    1632
r      72
u      48
b      48
o      48
y      48
Name: spore-print-color, dtype: int64

In [395]:
dataset['population'].value_counts()

v    4040
y    1712
s    1248
n     400
a     384
c     340
Name: population, dtype: int64

In [396]:
dataset['habitat'].value_counts()

d    3148
g    2148
p    1144
l     832
u     368
m     292
w     192
Name: habitat, dtype: int64

Проанализировав полученные данные можно заметить большое количество несбилансированных классов, а именно:
* gill-attachment
* gill-spacing
* stalk-color-above-ring
* stalk-color-below-ring
* veil-color

Объясняется это распростронением грибов в природе, некоторые виды встречаются гораздо чаще чем другие. Так же данный датасет не включает в себя абсолютно все виды грибов, а только 23 из них, что несколько уменьшает значимость получаемых на выходе данных о съедобности гриба, так как если нам на вход попадётся разновидность гриба не входившего в список датасета, возможен неправильный результат.
# 3. Обработка пропущенных значений

In [397]:
dataset.isna().sum()

class                       0
cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64

Данный датасет не имеет пропущенных значений.
# 4. Обработка категориальных значений
Бинарные признаки заменим на числовые 0 и 1.

In [398]:
dataset['class'] = pd.factorize(dataset['class'])[0]


In [399]:
dataset['bruises'] = pd.factorize(dataset['bruises'])[0]

In [400]:
dataset['gill-size'] = pd.factorize(dataset['gill-size'])[0]

In [401]:
dataset['stalk-shape'] = pd.factorize(dataset['stalk-shape'])[0]

Для остальных признаков применим метод бинаризации (one-hot encoding). В итоге таблица становится гигантских размеров и крайне сложной для визуализации.

In [402]:
capShape_dummies = pd.get_dummies(dataset['cap-shape'])
capShape_dummies.head()

,b,c,f,k,s,x
0,0,0,0,0,0,1
1,0,0,0,0,0,1
2,1,0,0,0,0,0
3,0,0,0,0,0,1
4,0,0,0,0,0,1


In [403]:
dataset = pd.concat((dataset, capShape_dummies), axis=1)
dataset = dataset.drop(['cap-shape'], axis=1)

In [404]:
capsurface_dummies = pd.get_dummies(dataset['cap-surface'])
dataset = pd.concat((dataset, capsurface_dummies), axis=1)
dataset = dataset.drop(['cap-surface'], axis=1)

In [405]:
capcolor_dummies = pd.get_dummies(dataset['cap-color'])
dataset = pd.concat((dataset, capcolor_dummies), axis=1)
dataset = dataset.drop(['cap-color'], axis=1)

In [406]:
odor_dummies = pd.get_dummies(dataset['odor'])
dataset = pd.concat((dataset, odor_dummies), axis=1)
dataset = dataset.drop(['odor'], axis=1)

In [407]:
gillattachment_dummies = pd.get_dummies(dataset['gill-attachment'])
dataset = pd.concat((dataset, gillattachment_dummies), axis=1)
dataset = dataset.drop(['gill-attachment'], axis=1)

In [408]:
gillspacing_dummies = pd.get_dummies(dataset['gill-spacing'])
dataset = pd.concat((dataset, gillspacing_dummies), axis=1)
dataset = dataset.drop(['gill-spacing'], axis=1)

In [409]:
gillcolor_dummies = pd.get_dummies(dataset['gill-color'])
dataset = pd.concat((dataset, gillcolor_dummies), axis=1)
dataset = dataset.drop(['gill-color'], axis=1)

In [410]:
stalkroot_dummies = pd.get_dummies(dataset['stalk-root'])
dataset = pd.concat((dataset, stalkroot_dummies), axis=1)
dataset = dataset.drop(['stalk-root'], axis=1)

In [411]:
stalksurfaceabovering_dummies = pd.get_dummies(dataset['stalk-surface-above-ring'])
dataset = pd.concat((dataset, stalksurfaceabovering_dummies), axis=1)
dataset = dataset.drop(['stalk-surface-above-ring'], axis=1)

In [412]:
stalksurfacebelowring_dummies = pd.get_dummies(dataset['stalk-surface-below-ring'])
dataset = pd.concat((dataset, stalksurfacebelowring_dummies), axis=1)
dataset = dataset.drop(['stalk-surface-below-ring'], axis=1)

In [413]:
stalkcolorabovering_dummies = pd.get_dummies(dataset['stalk-color-above-ring'])
dataset = pd.concat((dataset, stalkcolorabovering_dummies), axis=1)
dataset = dataset.drop(['stalk-color-above-ring'], axis=1)

In [414]:
stalkcolorbelowring_dummies = pd.get_dummies(dataset['stalk-color-below-ring'])
dataset = pd.concat((dataset, stalkcolorbelowring_dummies), axis=1)
dataset = dataset.drop(['stalk-color-below-ring'], axis=1)

In [415]:
veilcolor_dummies = pd.get_dummies(dataset['veil-color'])
dataset = pd.concat((dataset, veilcolor_dummies), axis=1)
dataset = dataset.drop(['veil-color'], axis=1)

In [416]:
ringtype_dummies = pd.get_dummies(dataset['ring-type'])
dataset = pd.concat((dataset, ringtype_dummies), axis=1)
dataset = dataset.drop(['ring-type'], axis=1)

In [417]:
sporeprintcolor_dummies = pd.get_dummies(dataset['spore-print-color'])
dataset = pd.concat((dataset, sporeprintcolor_dummies), axis=1)
dataset = dataset.drop(['spore-print-color'], axis=1)

In [418]:
population_dummies = pd.get_dummies(dataset['population'])
dataset = pd.concat((dataset, population_dummies), axis=1)
dataset = dataset.drop(['population'], axis=1)

In [419]:
habitat_dummies = pd.get_dummies(dataset['habitat'])
dataset = pd.concat((dataset, habitat_dummies), axis=1)
dataset = dataset.drop(['habitat'], axis=1)

# 5. Нормализация количественных признаков
Выполним стандартизацию всех признаков.

In [420]:
data_numerical = dataset['ring-number']
data_numerical = (data_numerical - data_numerical.mean(axis = 0))/data_numerical.std(axis = 0)
dataset = dataset.drop(['ring-number'], axis=1)
dataset = pd.concat((dataset, data_numerical), axis=1)

# 6. Разбиеные данных
Все оставшиеся признаки необходими для классификации, так что разобьём нашу выборку на тестовую и обучающую.

In [421]:
dataset.head()

,class,bruises,gill-size,stalk-shape,b,c,f,k,s,x,...,v,y,d,g,l,m,p,u,w,ring-number
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,-0.28414
1,1,0,1,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,-0.28414
2,1,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,-0.28414
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,-0.28414
4,1,1,1,1,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,-0.28414


In [430]:
X = dataset.drop(['class'], axis = 1)
Y = dataset['class']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.5, random_state = 32)

N_train, _ = X_train.shape 
N_test,  _ = X_test.shape 

N_train, N_test

(4062, 4062)

# 7. Классификатор N ближайших соседей

Knn - метод работает странно

In [423]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
nnb = [1, 2, 3, 5, 10, 15, 20, 30, 40, 50]
knn = KNeighborsClassifier()
grid = GridSearchCV(knn, param_grid = {'n_neighbors': nnb}, cv=10)
grid.fit(X_train, y_train)

best_cv_err = 1 - grid.best_score_
best_n_neighbors = grid.best_estimator_.n_neighbors
print(best_cv_err, best_n_neighbors)

0.0 1


In [436]:
knn = KNeighborsClassifier(n_neighbors = best_n_neighbors).fit(X_train, y_train)

err_train = np.mean(y_train != knn.predict(X_train))
err_test  = np.mean(y_test  != knn.predict(X_test))

print(err_train, err_test)

0.0 0.0


Ошибки по какой то причине нет!

# 9. SVC машина опорных векторов

In [432]:
from sklearn.svm import SVC
svc = SVC(gamma='auto')
svc.fit(X_train, y_train)

err_train = np.mean(y_train != svc.predict(X_train))
err_test  = np.mean(y_test  != svc.predict(X_test))

print(err_train, err_test)

0.003200393894633186 0.004677498769079271


# 10. Random forest

In [437]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier(n_estimators = 1000)
rf.fit(X_train, y_train)

err_train = np.mean(y_train != rf.predict(X_train))
err_test  = np.mean(y_test  != rf.predict(X_test))

print(err_train, err_test)

0.0 0.0


# 11. AdaBoost

In [438]:
ada = ensemble.AdaBoostClassifier(n_estimators = 1000)
ada.fit(X_train, y_train)

err_train = np.mean(y_train != ada.predict(X_train))
err_test = np.mean(y_test != ada.predict(X_test))

print(err_train, err_test)

0.0 0.0009847365829640572


# 12. GBT

In [439]:
gbt = ensemble.GradientBoostingClassifier(n_estimators = 1000)
gbt.fit(X_train, y_train)

err_train = np.mean(y_train != gbt.predict(X_train))
err_test = np.mean(y_test != gbt.predict(X_test))

print(err_train, err_test)

0.0 0.0009847365829640572


# 13. Вывод 
Почти все методы не дают ошибки, возможно есть ошибка в предидущих действиях, либо возможно существуют значимые категориальные признаки с помощью которых с высокой вероятностью можно определить съедобен ли гриб.